1. Using scipy to import channel data into python

In [44]:
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

In [45]:
matA = scipy.io.loadmat('./data/2018.10.29_18.57.37_1.A.mat')
matB = scipy.io.loadmat('./data/2018.10.29_18.57.37_1.B.mat')

# 1. Zero Noise threshold of Data

In [46]:
matA['T1'].shape

(1, 16777216)

In [47]:
# This is the filtering Channel
x1 = matB['T1'][0,:]
y1 = matB['Y1'][0,:]

In [49]:
#%matplotlib inline

plt.plot(x1[:100000],y1[:100000])
plt.show(block=True)

We assume the noise threshold is ???, if you decrease this, you'll see a sharp peak in the histogram in the next cell

In [50]:
%matplotlib qt

zero_level = 7950
#we are assuming the noise threshold is at 8150
yt_filter = np.clip(y1, a_min=zero_level,a_max=None)
yt_filter -= zero_level
#decrese everything by noise threshold, clip to 0

plt.plot(x1,yt_filter)
plt.show(block=True)

In [51]:
from scipy.signal import find_peaks

filter_peaks, _ = find_peaks(yt_filter, height=0,distance=15) #peaks are the x indices

plt.figure()
# plot graph with detected peaks over it
plt.plot(x1,yt_filter)
plt.plot(x1[filter_peaks], yt_filter[filter_peaks], "x")

#formatting
plt.xticks(rotation='vertical')
plt.xlabel("Time (s)")
plt.show(block=True)

plt.figure()
plt.hist(yt_filter[filter_peaks], bins=1500, log=False,histtype='step')
plt.show(block=True)


We can set the regime of one photon as from 0-250, 2 photos as 250-500 and 3 photons as 500-750. Eventually might want more sophisticated analysis
*** USE ABOVE HISTOGRAM TO ANALYZE ***

In [36]:
photon_ranges = [60, 170]

single_peaks = []
double_peaks = []
triple_peaks = []
for i in range(len(filter_peaks)):
    if yt_filter[filter_peaks[i]] < photon_ranges[0]:
        single_peaks.append(filter_peaks[i])
    elif yt_filter[filter_peaks[i]] < photon_ranges[1]:
        double_peaks.append(filter_peaks[i])
    else:
        triple_peaks.append(filter_peaks[i])

print("total peaks: ", len(filter_peaks))
print("single peaks: ", len(single_peaks))
print("double peaks: ", len(double_peaks))
print("triple peaks: ", len(triple_peaks))

filter_peaks = single_peaks

plt.figure()
plt.hist(yt_filter[filter_peaks], bins=500, log=False,histtype='step')
plt.show(block=True)
        
x2 = matA['T1'][0,:]
y2 = matA['Y1'][0,:]

#plt.plot(x2[:100000],y2[:100000])
#plt.show()

total peaks:  5384
single peaks:  1631
double peaks:  3726
triple peaks:  27


In [9]:
from scipy import stats
#we are assuming the noise threshold is at 8150
yt_opo = np.clip(y2, a_min=zero_level,a_max=None)
yt_opo -= zero_level
#decrese everything by noise threshold, clip to 0

#plt.plot(x2[:100000],ytB[:100000])
#plt.show()

In [10]:
from scipy.signal import find_peaks

opo_peaks, _ = find_peaks(yt_opo, height=0,distance=15) #peaks are the x indices

plt.figure()
# plot graph with detected peaks over it
plt.plot(x2,yt_opo)
plt.plot(x2[opo_peaks], yt_opo[opo_peaks], "x")

#formatting
plt.xticks(rotation='vertical')
#plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.xlabel("Time (s)")
#plt.show(block=True)

plt.figure()
plt.hist(yt_opo[opo_peaks], bins=1500, log=False,histtype='step')
#plt.ylim(0,25)
plt.show(block=True)

# 2. Let's try to calculate the Heralding Ratio

Heralding Ratio = coincidences/total peaks for all photon amounts

In [11]:
#would want to use a set data structure for B for faster run-time
def peakInRange(peakI, peaksA,rng):
    for i in range(-rng,rng+1):
        if (peakI+i in peaksA):
            return True
    return False
        
def findRatioInRange(rng,filter_peaks, opo_peaks):
    coincidences= 0
    for peakI in filter_peaks:
        if peakInRange(peakI,opo_peaks,rng):
            coincidences+=1
    return coincidences/filter_peaks.shape[0]

#heralding ratios for different windows
ratios = []
for i in range (0,5):
    ratios.append(findRatioInRange(i, filter_peaks, opo_peaks))
    print("window: "+ str(i), " ratio: " + str(ratios[i]))

window: 0  ratio: 0.00687221396731055
window: 1  ratio: 0.02526002971768202
window: 2  ratio: 0.09528231797919762
window: 3  ratio: 0.23736998514115898
window: 4  ratio: 0.3138930163447251


For 2017.10.18_17.10.19_1 data

window: 0  ratio: 0.08497961743493258
window: 1  ratio: 0.21730950141110067
window: 2  ratio: 0.3101285669488868
window: 3  ratio: 0.3759799310128567
window: 4  ratio: 0.4179993728441518
window: 5  ratio: 0.455942301661963
window: 6  ratio: 0.4825964252116651
window: 7  ratio: 0.5152085293195359
window: 8  ratio: 0.5453120100344936
window: 9  ratio: 0.5678896205707118
window: 10  ratio: 0.5876450297899027
window: 11  ratio: 0.6095954844778928
window: 12  ratio: 0.6284101599247413
window: 13  ratio: 0.6412668548134212
window: 14  ratio: 0.6647851991219819
window: 15  ratio: 0.6792097836312324
window: 16  ratio: 0.6895578551269991
window: 17  ratio: 0.6973973032298526
window: 18  ratio: 0.7105675760426466

window: 19  ratio: 0.7234242709313264

For 2018.10.29_18.57.37_1 data

window: 0  ratio: 0.00687221396731055
window: 1  ratio: 0.02526002971768202
window: 2  ratio: 0.09528231797919762
window: 3  ratio: 0.23736998514115898
window: 4  ratio: 0.3138930163447251
window: 5  ratio: 0.3385958395245171
window: 6  ratio: 0.35085438335809804
window: 7  ratio: 0.3638558692421991
window: 8  ratio: 0.3725854383358098
window: 9  ratio: 0.3835438335809807
window: 10  ratio: 0.39561664190193163
window: 11  ratio: 0.4062035661218425
window: 12  ratio: 0.4156760772659733
window: 13  ratio: 0.42329123328380386
window: 14  ratio: 0.4312778603268945
window: 15  ratio: 0.4414933135215453
window: 16  ratio: 0.45170876671619614
window: 17  ratio: 0.4606240713224368
window: 18  ratio: 0.4723254086181278
window: 19  ratio: 0.4810549777117385

# 3. Heralding Ratio with Multiplied Channels

In [18]:
mulChannel = yt_filter * yt_opo
plt.plot(x1,mulChannel)
# plt.show(block=True)

mul_peaks, _ = find_peaks(mulChannel, height=0,distance=15) #peaks are the x indices

plt.figure()
plt.hist(mulChannel[mul_peaks], bins=500, log=False,histtype='step')
plt.show(block=True)

In [19]:
print("Heralding Ratio: " + str(len(mul_peaks)/len(filter_peaks)))

Heralding Ratio: 0.44483655274888556
